![astropy logo](http://astropy.readthedocs.org/en/stable/_images/astropy_banner.svg)

<br /><br />

# [`astroplan`](https://astroplan.readthedocs.org/en/latest/) <a id='astroplan'></a>


`astroplan` is an `astropy`-affiliated package that helps you calculate when objects are observable from observatories on the Earth (Morris et al. 2017, submitted).


## Table of contents
* [astroplan](#astroplan)
  * [Intro to astroplan](#introastroplan)
  * [Constraints](#constraints)
  * [Plots](#plots)



<br /><br />

### Intro to astroplan <a id='introastroplan'></a>

You can describe an observatory using the [`Observer`](https://astroplan.readthedocs.io/en/latest/api/astroplan.Observer.html#astroplan.Observer) object, which knows about famous observatories:

In [ ]:
from astroplan import Observer

# Use the `at_site` method to access famous observatories
salt = Observer.at_site('SALT')
keck = Observer.at_site('Keck')

from astropy.coordinates import EarthLocation
print("Available observatories: \n\n{0}"
      .format(', '.join(EarthLocation.get_site_names())))

You can also specify observatories not included in the database using an [`EarthLocation`](http://docs.astropy.org/en/stable/api/astropy.coordinates.EarthLocation.html) object:

In [ ]:
from astropy.coordinates import EarthLocation

latitude = -33.9249*u.deg
longitude = 18.4241*u.deg
elevation = 0*u.m

# Specify the location of Cape Town:
location = EarthLocation.from_geodetic(longitude, latitude, elevation)

# Make an observer at Cape Town:
cape_town = Observer(location=location, pressure=1*u.bar, temperature=20*u.deg_C)

cape_town

The pressure and temperature will be used in calculations for atmospheric refraction when computing the altitude and azimuth of targets.

We define a target in the sky with a `FixedTarget` object, which simply contains a `SkyCoord` and a name:

In [ ]:
from astroplan import FixedTarget
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra='14h29m42.94853s', dec='-62d40m46.1631s', 
                 distance=4.224*u.lightyear)
proxima = FixedTarget(coord=coord, name='Proxima Cen')

We can use the `from_name` method to grab targets with their identifier, resolved via [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame):

In [ ]:
# Targets are stored as `astroplan.FixedTarget` objects
target_names = ['Proxima', 'LMC', 'SMC', 'alpha Mensa', 'Polaris']
targets = [FixedTarget.from_name(target) for target in target_names]

Let's see which of these targets is "up" ($>0^\circ$ altitude) now:

In [ ]:
from astropy.time import Time

# Which targets are visible right now?
cape_town.target_is_up(Time.now(), targets)

The circumpolar souther stars are up, and the circumpolar northern stars are not up.

<br />
*** 
<br />

### [Constraints](http://astroplan.readthedocs.io/en/latest/tutorials/constraints.html) <a id='constraints'></a>

Often you will need to plan observations given various constraints, for example, you might need to observe: 
* Between civil twilights (sun at $<-6^\circ$ altitude)
* The target altitude must be $20^\circ < $alt$ < 85^\circ$ (or similar pointing limits for your telescope)
* The target is separated from the moon by at least $50^\circ$

`astroplan` provides a framework for defining these observing constraints, and computing whether or not targets are observable given those constraints. Let's implement the above list of constraints: 

In [ ]:
from astroplan import (AtNightConstraint, AltitudeConstraint, 
                       MoonSeparationConstraint, MoonIlluminationConstraint)

constraints = [AtNightConstraint.twilight_civil(),
               AltitudeConstraint(min=20*u.deg, max=85*u.deg), 
               MoonSeparationConstraint(min=2*u.deg)]

In [ ]:
def plot_constraints(observability_grid, constraints, time_grid, name):
    """
    Plot some constraints in a grid!
    """
    extent = [-0.5, -0.5+len(time_grid), -0.5,  -0.5+len(constraints)]

    fig, ax = plt.subplots()
    ax.imshow(observability_grid, extent=extent)

    ax.set_yticks(range(0, len(constraints)))
    ax.set_yticklabels([c.__class__.__name__ for c in constraints])

    ax.set_xticks(range(len(time_grid)))
    ax.set_xticklabels([t.datetime.strftime("%H:%M") for t in time_grid])

    ax.set_xticks(np.arange(extent[0], extent[1]), minor=True)
    ax.set_yticks(np.arange(extent[2], extent[3]), minor=True)

    ax.grid(which='minor', color='w', linestyle='-', linewidth=2)
    ax.tick_params(axis='x', which='minor', bottom='off')
    plt.setp(ax.get_xticklabels(), rotation=30, ha='right')

    ax.tick_params(axis='y', which='minor', left='off')
    ax.set_xlabel('Time on {0} UTC'.format(time_grid[0].datetime.date()))
    ax.set_title(name)
    fig.subplots_adjust(left=0.35, right=0.9, top=0.9, bottom=0.1)
    return fig, ax

In [ ]:
from astroplan.utils import time_grid_from_range

# Define range of times to observe between
start_time = Time('2017-11-14 22:00:01')
end_time = Time('2017-11-15 04:00:01')
time_resolution = 1 * u.hour

# Create grid of times from ``start_time`` to ``end_time``
# with resolution ``time_resolution``
time_grid = time_grid_from_range([start_time, end_time],
                                 time_resolution=time_resolution)

for target in targets: 
    observability_grid = np.zeros((len(constraints), len(time_grid)))

    for i, constraint in enumerate(constraints):
        # Evaluate each constraint
        observability_grid[i, :] = constraint(cape_town, target, times=time_grid)
    
    plot_constraints(observability_grid, constraints, time_grid, target.name)

In the above plot, grid squares that are purple represent times and constraints for which the target is observable, yellow is not observabable.

Another way to visualize whether or not a target meets the constraints is with the `observability_table` function:

In [ ]:
from astroplan import observability_table
observability_table(constraints, cape_town, targets, times=time_grid)

### Plots <a id='plots'></a>

Let's track that target's motion through the sky for the next ten hours in a plot: 

In [ ]:
from astroplan.plots import plot_sky

# Plot at times: 
plot_times = Time.now() + np.linspace(0, 10, 10)*u.hour

for target in targets:
    plot_sky(target, cape_town, plot_times)
plt.legend(loc=[1.2, 0])

Perhaps you need to make a finder chart to help you find Proxima:

In [ ]:
from astroplan.plots import plot_finder_image

plot_finder_image(targets[0], fov_radius=5*u.arcmin, 
                  reticle=True, survey='2MASS-K')